In [ ]:
import tensorflow as tf
import time
import matplotlib
from matplotlib import pyplot
%run lstm.ipynb

In [ ]:
# Гиперпараметры.
num_epochs = 30
num_repeat = 15
input_dim = 5
hidden_dim = 7
learn_rate = 0.5
pruning_iters = 20
pruning_strength = 0.5
optimizer = 'adagrad'

# sine, ascending-sine
data_sequence = 'sine'
normalize = True

In [ ]:
def split_data(data, ratio):
    num_samples = int(len(data) * ratio)
    return data[:num_samples], data[num_samples:]

In [ ]:
def shift(seq, seq_size, step=1):
    x, y = [], []
    for i in np.arange(0, len(seq) - seq_size - step + 1, step):
        xc = seq[i:i+seq_size]
        yc = seq[i+step:i+seq_size+step]
        x.append(xc)
        y.append(yc)
    return x, y

In [ ]:
def train(net, x_train, y_train):
    begin = time.time()

    for e in range(num_epochs):
        net.clear_state()
        for x, y in zip(x_train, y_train):
            net.train([x], [y], repeat=num_repeat)

    return time.time() - begin

In [ ]:
def predict(net, start, length):
    seq_prev = [start]
    seq_pred = []

    for i in range(length):
        seq_prev = net.test(seq_prev)
        seq_pred.append(seq_prev[0])

    # Выделяем из предсказанной последовательности нужную нам точку.
    seq_points = []
    
    for i in seq_pred:
        seq_points.append(i[-1])
        
    return seq_points

In [ ]:
x = np.linspace(0., 20. * np.pi, 200)

# Здесь работает переключение способа генерации данных для обработки.
if data_sequence == 'sine':
    seq = np.sin(x)
elif data_sequence == 'ascending-sine':
    seq =  np.sin(x) + 0.1 * x + 0.5

if normalize == True:
    seq_max = np.amax(seq)
    seq /= seq_max

In [ ]:
seq_train, seq_test = split_data(seq, 0.3)
x_train, y_train = shift(seq_train, input_dim, step=1)

matplotlib.pyplot.plot(np.arange(0., len(seq_train), 1.), seq_train)
matplotlib.pyplot.plot(np.arange(len(seq_train), len(seq_train) + len(seq_test), 1.), seq_test)
#matplotlib.pyplot.plot(np.linspace(len(x_train), len(x_train) + len(x_test), len(x_test)), x_test)

In [ ]:
sess = tf.Session()
net = lstm_cell(sess, input_dim, hidden_dim, learn_rate, optimizer)

In [ ]:
# Различные интересные зависимости.
c_mae = []
c_mse = []
c_norm = []
c_train_time = []

In [ ]:
for i in range(pruning_iters):
    
    # Обучение ячейки на последовательности.
    train_time = train(net, x_train, y_train)

    # Сохранить состояние сети.
    net.save_all()

    # Предсказание.
    #seq_points = predict(net, x_train[-1], len(seq_test))
    seq_points = predict(net, y_train[-1], len(seq_test))

    # Считаем ошибки предсказания.
    mae = np.sum(np.abs(seq_points - seq_test)) / len(seq_points)
    mse = np.sum(np.square(seq_points - seq_test)) / len(seq_points)
    
    if i > 2 and mae >= c_mae[-1]:
        print('SATURATION!')
        break

    # После того как последовательность была предсказана стоит восстановить состояние ячейки.
    net.restore_all()
    
    # Делаем прореживание. Сохраняем значение насколько матрицы сохраняют своё схожесть
    # после прореживания с исходными матрицами.
    norm, _ = net.svd_compress(pruning_strength, pruning_strength)
    
    c_mae.append(mae)
    c_mse.append(mse)
    c_norm.append(norm)
    c_train_time.append(train_time)
    
    print('iter', i, 'train_time', train_time, 'norm', norm, 'mae', mae, 'mse', mse)
    
net.restore_all()

https://matplotlib.org/devdocs/gallery/subplots_axes_and_figures/subplots_demo.html

In [ ]:
fig, ax = matplotlib.pyplot.subplots(2, 2)

ax[0,0].set_title('Время обучения, с.')
ax[0,0].plot(c_train_time)

ax[0,1].set_title('Норма SVD разложения')
ax[0,1].plot(c_norm)

ax[1,0].set_title('MAE')
ax[1,0].plot(c_mae)

ax[1,1].set_title('MSE')
ax[1,1].plot(c_mse)

In [ ]:
fig, ax = matplotlib.pyplot.subplots(2, 2)

ax[0,0].set_title('Ft Wx')
#ax[0,0].spy(net.sess.run(net.ft_wx))
ax[0,0].matshow(net.sess.run(net.ft_wx))

ax[0,1].set_title('It Wx')
ax[0,1].matshow(net.sess.run(net.it_wx))

ax[1,0].set_title('Ctt Wx')
ax[1,0].matshow(net.sess.run(net.ctt_wx))

ax[1,1].set_title('Ot Wx')
ax[1,1].matshow(net.sess.run(net.ot_wx))

In [ ]:
seq_points = predict(net, y_train[-1], len(seq_test))
net.restore_all()

In [ ]:
# Отображаем график.
matplotlib.pyplot.plot(seq_test, 'orange')
#matplotlib.pyplot.plot(seq_points)
matplotlib.pyplot.plot(seq_points, '--')

# Отображаем статистику обучения.
print('optimizer', optimizer)
print('learn_rate', learn_rate)
print('train_time', train_time)
print('num_epochs', num_epochs)
print('num_repeat', num_repeat)
print('input_dim', input_dim)
print('hidden_dim', hidden_dim)
print('MAE', c_mae[-1])
print('MSE', c_mse[-1])